# Trabalho Pratico 2 - Cifra A5/1
## Grupo 27
### LCC 2024/2025
#### Rafaela Antunes Pereira A102527
#### Gonçalo Gonçalves Barroso A102931
#### Ricardo Eusebio Cerqueira A102878


### Explicação do problema a)
Considerando a descrição da cifra A5/1, pretende-se definir e codificar, em Z3 e usando o tipo BitVec para modelar a informação, uma FSM que descreva o gerador de chaves.


In [146]:
from z3 import *
import random

def declare(i):
    return {
        'LFSR1': BitVec(f'LFSR1{i}', 19),
        'LFSR2': BitVec(f'LFSR2{i}', 22),
        'LFSR3': BitVec(f'LFSR3{i}', 23),
        'keys': BitVec(f'keys_{i}', 64),  # Vetor de 64 bits
        'kepos': BitVec(f'kepos_{i}', 7)
        
        
    }

def init(state):
    z0 = random.getrandbits(19)
    z1 = random.getrandbits(22)
    z2 = random.getrandbits(23)

    return And(
        state['LFSR1'] == BitVecVal(z0, 19),
        state['LFSR2'] == BitVecVal(z1, 22),
        state['LFSR3'] == BitVecVal(z2, 23),
        state['keys'] == BitVecVal(0, 64),  # Inicializado com zero
        state['kepos'] == BitVecVal(0, 7) 
    )

def avanca(curr, bits_tapped):
    feed_bits = [Extract(i, i, curr) for i in bits_tapped]
    feedback = BitVecVal(0, 1)
    for bit in feed_bits:
        feedback ^= bit
    LFSR_deslocado = curr >> 1
    feed_deslocado = ZeroExt(curr.size() - 1, feedback) << (curr.size() - 1)
    return LFSR_deslocado | feed_deslocado

def maioria_bit(control_1, control_2, control_3):
    return (control_1 & control_2) | (control_1 & control_3) | (control_2 & control_3)

def trans(curr, prox):
    control_1 = Extract(8, 8, curr['LFSR1'])
    control_2 = Extract(10, 10, curr['LFSR2'])
    control_3 = Extract(10, 10, curr['LFSR3'])
    maioria = maioria_bit(control_1, control_2, control_3)
    bits_tapped1 = [13, 16, 17, 18]
    bits_tapped2 = [20, 21]
    bits_tapped3 = [7, 20, 21, 22]

    next_LFSR1 = If(maioria == control_1, avanca(curr['LFSR1'], bits_tapped1), curr['LFSR1'])
    next_LFSR2 = If(maioria == control_2, avanca(curr['LFSR2'], bits_tapped2), curr['LFSR2'])
    next_LFSR3 = If(maioria == control_3, avanca(curr['LFSR3'], bits_tapped3), curr['LFSR3'])

    new_bit = Extract(0, 0, next_LFSR1) ^ Extract(0, 0, next_LFSR2) ^ Extract(0, 0, next_LFSR3)
    updated_keys = (curr['keys'] << 1) | ZeroExt(63, new_bit)
    updated_pos = curr['kepos'] + 1

    return And(
        prox['LFSR1'] == next_LFSR1,
        prox['LFSR2'] == next_LFSR2,
        prox['LFSR3'] == next_LFSR3,
        prox['keys'] == updated_keys,
        prox['kepos'] == updated_pos,
        
    )



                                                                        # (posições que estão a uma distância de no máximo 2 ** (t // 2) de i)

def print_state(i, model, state):
    LFSR1val = model[state['LFSR1']].as_long()
    LFSR2val = model[state['LFSR2']].as_long()
    LFSR3val = model[state['LFSR3']].as_long()
    keysval = model[state['keys']].as_long()
    posval = model[state['kepos']].as_long()

    print(f"\nEstado {i}:")
    print(f"LFSR 0 - {bin(LFSR1val)[2:].zfill(19)}")
    print(f"LFSR 1 - {bin(LFSR2val)[2:].zfill(22)}")
    print(f"LFSR 2 - {bin(LFSR3val)[2:].zfill(23)}")
    print(f"Output -  {bin(keysval)[2:].zfill(64)}")
    
                    

def main(declare, init, trans, k, t):
    states = [declare(i) for i in range(k + 1)]
    solver = Solver()
    solver.add(init(states[0]))
    
    for i in range(k):
        solver.add(trans(states[i], states[i + 1]))
        if solver.check() == sat:
            model = solver.model()
            print_state(i, model, states[i])
           

# Execução com os parâmetros k e t especificados
print("\nExemplo com k = 64 e t = 4")
k = 64
t = 3
main(declare, init, trans, k, t)

print("\nExemplo com k = 10 e t = 3")
k = 10
t = 3
main(declare, init, trans, k, t)




Exemplo com k = 64 e t = 4

Estado 0:
LFSR 0 - 0010001011011110101
LFSR 1 - 1001101000110001010100
LFSR 2 - 10110110010110010011100
Output -  0000000000000000000000000000000000000000000000000000000000000000

Estado 1:
LFSR 0 - 0010001011011110101
LFSR 1 - 1100110100011000101010
LFSR 2 - 11011011001011001001110
Output -  0000000000000000000000000000000000000000000000000000000000000001

Estado 2:
LFSR 0 - 0010001011011110101
LFSR 1 - 1110011010001100010101
LFSR 2 - 11101101100101100100111
Output -  0000000000000000000000000000000000000000000000000000000000000011

Estado 3:
LFSR 0 - 1001000101101111010
LFSR 1 - 1111001101000110001010
LFSR 2 - 11110110110010110010011
Output -  0000000000000000000000000000000000000000000000000000000000000111

Estado 4:
LFSR 0 - 1100100010110111101
LFSR 1 - 1111001101000110001010
LFSR 2 - 11111011011001011001001
Output -  0000000000000000000000000000000000000000000000000000000000001110

Estado 5:
LFSR 0 - 1100100010110111101
LFSR 1 - 1111100110100011000101
